# 5.3 Lab: Cross-Validation and the Bootstrap

## 5.3.1 The Validation Set Approach

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd 
import math
import random

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model

/Users/qiuping1/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
import pandas as pd 
Auto = pd.read_csv('data/Auto.csv', header=0, na_values='?')
Auto = Auto.dropna().reset_index(drop=True) # drop the observation with NA values and reindex the obs from 0
Auto.shape

(392, 9)

### Python and R use different random number generator, so we may see slightly difference results in this chapter

In [4]:
np.random.seed(1)
train = np.random.choice(Auto.shape[0], 196, replace=False)
select = np.in1d(range(Auto.shape[0]), train)

In [5]:
import statsmodels.formula.api as smf
lm = smf.ols ('mpg~horsepower', data = Auto[select]).fit()
print lm.summary()
preds = lm.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print '--------Test Error for 1st order--------'
print np.mean(square_error[~select])

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Fri, 21 Apr 2017   Prob (F-statistic):           1.28e-42
Time:                        16:48:18   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.0

In [6]:
lm2 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0)', data = Auto[select]).fit()
preds = lm2.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print '--------Test Error for 2nd order--------'
print np.mean(square_error[~select])

--------Test Error for 2nd order--------
20.2526908584


In [7]:
lm3 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0) + I(horsepower ** 3.0)', data = Auto[select]).fit()
preds = lm3.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print '--------Test Error for 3rd order--------'
print np.mean(square_error[~select])

--------Test Error for 3rd order--------
20.3256093659


### These results are consistent with our previous findings: a model that predicts mpg using a quadratic function of horsepower performs better than a model that involves only a linear function of horsepower, and there is little evidence in favor of a model that uses a cubic function of horsepower.

### If we look at the summmary for 3rd order regression, the coefficient of the 3rd order term is not statistically significant. I will use this as Supporting evidence for the above claim. 

In [8]:
print lm3.summary()

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     165.9
Date:                Fri, 21 Apr 2017   Prob (F-statistic):           4.60e-53
Time:                        16:48:18   Log-Likelihood:                -561.56
No. Observations:                 196   AIC:                             1131.
Df Residuals:                     192   BIC:                             1144.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               66.5200 

## 5.3.2 Leave-One-Out Cross-Validation

### OLS Fit

In [9]:
ols_fit = smf.ols ('mpg~horsepower', data = Auto).fit()
print ols_fit.params

Intercept     39.935861
horsepower    -0.157845
dtype: float64


### GLM Fit. Compare with OLS fit, the coeffs are the same

In [10]:
glm_fit = sm.GLM.from_formula('mpg~horsepower', data = Auto).fit()
print glm_fit.params

Intercept     39.935861
horsepower    -0.157845
dtype: float64


### Trying CV in Python is not as easy as that in R. It will require some manual coding.

### To use some of implemented function in Python, we use Sklearn for linear model 

In [11]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [12]:
x = pd.DataFrame(Auto.horsepower)
y = Auto.mpg

model = LinearRegression()
model.fit(x, y)
print model.intercept_
print model.coef_

39.9358610212
[-0.15784473]


In [13]:
k_fold = KFold(n_splits=x.shape[0]) # loo use folds equal to # of observations
test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
print np.mean(-test)

24.2315135179


### For higher order polynomial fit, we use pipline tool. Below shows how to fit an order 1 to 5 polynomial data and show the loo results

In [14]:
A = []
for porder in xrange(1, 6):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=x.shape[0]) # loo use folds equal to # of observations
    test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
    A.append(np.mean(-test))
    
print A

[24.231513517929226, 19.248213124489389, 19.334984064114092, 19.424430308545745, 19.033219754727607]


## 5.3.3 k-Fold Cross-Validation

### K-fold validation is exactly same as LOO with different n_splits parameter setup. The computation time is much shorter than that of LOOCV.

In [15]:
np.random.seed(2)
A = []
for porder in xrange(1, 11):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=10) 
    test = cross_val_score(model, x, y, cv = k_fold,  scoring = 'neg_mean_squared_error', n_jobs = -1)
    A.append(np.mean(-test))
    
print A

[27.439933652339857, 21.235840055802118, 21.336606183328417, 21.35388698756352, 20.905633737044905, 20.78270442749729, 20.953103378424785, 21.077131628861984, 21.036781313639977, 20.980956456366364]


### We still see little evidence that using cubic or higher-order polynomial terms leads to lower test error than simply using a quadratic fit.

## 5.3.4 The Bootstrap

### Bootstrap means sampling with replacement. To eliminate the effect of sample size, the norm practice is to sample the same size as original dataset with replacement.

In [16]:
Portfolio = pd.read_csv('data/Portfolio.csv', header=0)

### To illustrate the use of the bootstrap on this data, we must first create a function, alpha_fn(), which takes as input the (X, Y) data as well as a vector indicating which observations should be used to estimate alpha.

In [32]:
def alpha_fn(data, index):
    X = data.X[index]
    Y = data.Y[index]
    return (np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0,1])

In [33]:
alpha_fn(Portfolio, range(0, 100))

0.57665115161041181

### Generate one set of random index with 100 elements. The array has been sorted to show there are repeat elements.

In [34]:
np.sort(np.random.choice(range(0, 100), size=100, replace=True))

array([ 1,  2,  2,  3,  4,  5,  5,  9,  9, 10, 11, 11, 14, 15, 15, 17, 17,
       17, 18, 19, 20, 20, 21, 22, 24, 25, 26, 29, 29, 31, 31, 34, 35, 35,
       37, 37, 40, 41, 41, 43, 44, 44, 45, 50, 51, 51, 53, 54, 54, 54, 55,
       55, 55, 55, 59, 59, 60, 60, 62, 63, 63, 63, 63, 64, 65, 65, 65, 65,
       66, 67, 69, 74, 76, 76, 78, 80, 80, 80, 81, 82, 83, 83, 83, 84, 86,
       86, 86, 88, 89, 90, 91, 93, 94, 94, 95, 96, 96, 96, 97, 99])

### Recall the previous function with a random set of input. 

In [35]:
alpha_fn(Portfolio, np.random.choice(range(0, 100), size=100, replace=True))

0.53925386135127662

### Since I am not aware of boot similar function in python, I just define a ad hoc function called boot_python()

In [36]:
def boot_python(data, input_fun, iteration):
    n = Portfolio.shape[0]
    idx = np.random.randint(0, n, (iteration, n))
    stat = np.zeros(iteration)
    for i in xrange(len(idx)):
        stat[i] = input_fun(data, idx[i])
    
    return {'Mean': np.mean(stat), 'STD': np.std(stat)}
    

In [39]:
boot_python(Portfolio, alpha_fn, 1000)

{'Mean': 0.58079136982741419, 'STD': 0.092340012131034557}

### Similar idea can be used in a lot of other places
### Estimating the accuracy of a linear regression model coeffcients / Conduct non-parametric testing (permutation test) / Estimate some complicated probability 